<a href="https://colab.research.google.com/github/ksknysn/Coursera/blob/main/TensorFlowDeveloper/C2_W2_Lab_1_horse_v_human_augmentation_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
!gdown --id 1onaG42NZft3wCE1WH0GDEbUhu75fedP5
!gdown --id 1LYeusSEIiZQpwN-mthh5nKdA75VsKG1U

Downloading...
From: https://drive.google.com/uc?id=1onaG42NZft3wCE1WH0GDEbUhu75fedP5
To: /content/horse-or-human.zip
150MB [00:01, 112MB/s] 
Downloading...
From: https://drive.google.com/uc?id=1LYeusSEIiZQpwN-mthh5nKdA75VsKG1U
To: /content/validation-horse-or-human.zip
11.5MB [00:00, 100MB/s]


In [11]:
import os
import zipfile

local_zip = './horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./horse-or-human')

local_zip = './validation-horse-or-human.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('./validation-horse-or-human')

zip_ref.close()

In [12]:
# Directory with our training horse pictures
train_horse_dir = os.path.join('./horse-or-human/horses')

# Directory with our training human pictures
train_human_dir = os.path.join('./horse-or-human/humans')

# Directory with our training horse pictures
validation_horse_dir = os.path.join('./validation-horse-or-human/horses')

# Directory with our training human pictures
validation_human_dir = os.path.join('./validation-horse-or-human/humans')

In [13]:
print('total training horse images:', len(os.listdir(train_horse_dir)))
print('total training human images:', len(os.listdir(train_human_dir)))
print('total validation horse images:', len(os.listdir(validation_horse_dir)))
print('total validation human images:', len(os.listdir(validation_human_dir)))

total training horse images: 500
total training human images: 527
total validation horse images: 128
total validation human images: 128


In [14]:
import tensorflow as tf

In [15]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('horses') and 1 for the other ('humans')
    tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop
model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(rescale=1/255)

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        './horse-or-human/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 300x300
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow training images in batches of 128 using train_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        './validation-horse-or-human/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 300x300
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [17]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=100,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8)

Epoch 1/100
8/8 [==============================] - 10s 1s/step - loss: 0.7770 - accuracy: 0.5239 - val_loss: 0.6724 - val_accuracy: 0.5000
Epoch 2/100
8/8 [==============================] - 9s 1s/step - loss: 1.1760 - accuracy: 0.6475 - val_loss: 0.5699 - val_accuracy: 0.6797
Epoch 3/100
8/8 [==============================] - 8s 1s/step - loss: 0.6097 - accuracy: 0.7642 - val_loss: 0.5206 - val_accuracy: 0.8086
Epoch 4/100
8/8 [==============================] - 8s 1s/step - loss: 0.4714 - accuracy: 0.8109 - val_loss: 1.0934 - val_accuracy: 0.7773
Epoch 5/100
8/8 [==============================] - 8s 1s/step - loss: 0.3940 - accuracy: 0.8409 - val_loss: 4.7760 - val_accuracy: 0.5977
Epoch 6/100
8/8 [==============================] - 9s 1s/step - loss: 0.4382 - accuracy: 0.8743 - val_loss: 0.9038 - val_accuracy: 0.8594
Epoch 7/100
8/8 [==============================] - 8s 1s/step - loss: 0.2573 - accuracy: 0.9121 - val_loss: 1.1615 - val_accuracy: 0.7891
Epoch 8/100
8/8 [================

In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

#validation_datagen = ImageDataGenerator(rescale=1/255)
validation_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        './horse-or-human/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 300x300
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow training images in batches of 128 using train_datagen generator
validation_generator = validation_datagen.flow_from_directory(
        './validation-horse-or-human/',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 300x300
        batch_size=32,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [ ]:
history = model.fit(
      train_generator,
      steps_per_epoch=8,  
      epochs=100,
      verbose=1,
      validation_data = validation_generator,
      validation_steps=8)

Epoch 1/100
8/8 [==============================] - 30s 4s/step - loss: 2.3766 - accuracy: 0.7175 - val_loss: 1.7526 - val_accuracy: 0.5156
Epoch 2/100
8/8 [==============================] - 29s 4s/step - loss: 0.5597 - accuracy: 0.7108 - val_loss: 0.6038 - val_accuracy: 0.6992
Epoch 3/100
8/8 [==============================] - 29s 4s/step - loss: 0.5403 - accuracy: 0.7353 - val_loss: 0.5800 - val_accuracy: 0.6914
Epoch 4/100
8/8 [==============================] - 28s 4s/step - loss: 0.4616 - accuracy: 0.7798 - val_loss: 0.9978 - val_accuracy: 0.5664
Epoch 5/100
8/8 [==============================] - 29s 4s/step - loss: 0.4314 - accuracy: 0.7976 - val_loss: 0.8106 - val_accuracy: 0.6445
Epoch 6/100
8/8 [==============================] - 30s 4s/step - loss: 0.3293 - accuracy: 0.8486 - val_loss: 2.5409 - val_accuracy: 0.5000
Epoch 7/100
8/8 [==============================] - 29s 4s/step - loss: 0.5528 - accuracy: 0.7842 - val_loss: 0.9373 - val_accuracy: 0.5547
Epoch 8/100
8/8 [==========

summary: there is no improvement with augmenting... :( also, it is worse than without augmenting

i added the augmenting to the validation then there is a little improvement